In [ ]:
#  -------------------------------------------
"""
입출력 양식을 
준수해 주십시오

###  INPUT ###
import pandas as pd
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

{
    
    Write codes...
    
    Training model name : model
    ex) y_pred = model.predict(X_test)

}


### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)
"""

In [ ]:
###  INPUT ###
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

In [ ]:
#    Write codes...
# 날짜 데이터를 datetime 형식으로 변환
input_data['date1'] = pd.to_datetime(input_data['date'], format="%Y%m%d")

# inTp, outTp가 0.0인 것들 중앙값으로 채움
input_data["inTp"]=input_data['inTp'].replace(0,input_data['inTp'].median())
input_data["outTp"]=input_data['outTp'].replace(0,input_data['inTp'].median())


# '초장' 열이 0인 행을 찾아서 값을 채워줌
for i in range(1, len(input_data)):
    if input_data.at[i, 'pllnLt'] == 0:
        # 현재 주차의 날짜에서 7일 전 날짜 계산
        current_date = pd.to_datetime(input_data.at[i, 'date1'])
        last_week_start_date = current_date - pd.DateOffset(weeks=1) - pd.DateOffset(days=6)
        yesterday = current_date - pd.DateOffset(days=1)

        # 'last_week_start_date'부터 'yesterday'까지의 날 중에서 '초장' 값이 있는 최근 날짜의 '초장' 값을 가져오기
        recent_chojang = input_data[(input_data['date1'] >= last_week_start_date.strftime('%Y-%m-%d')) & (input_data['date1'] <= yesterday.strftime('%Y-%m-%d'))]['pllnLt'].max()

        # '초장' 열 값을 최근 '초장' 값 + 'grwtLt' 값으로 채워줌
        input_data.at[i, 'pllnLt'] = recent_chojang + input_data.at[i, 'grwtLt']

input_data['pllnLt'].fillna(0, inplace=True)

x = input_data[['date', 'outTp', 'frmWeek', 'cunt', 'outWs', 'acSlrdQy', 'hvstGrupp',
                'MistUsageTime', 'WaterUsage', 'FertilizerUsage','inTp', 'pllnLt']]
y = input_data[['outtrn_cumsum', 'HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = ExtraTreesRegressor(n_estimators=50)
model = MultiOutputRegressor(model)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

def calculate_rmse(targets, predictions):
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))

def calculate_R2_score(y_test, y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_test, y_pred)

rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

In [ ]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)
